# 1. Load Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode(connected=True)
pd.set_option('display.max_rows', 20)
import re

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

WHO_Life_Exp = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/WHOregionLifeExpectancyAtBirth.csv')  
WHO_HALE_EXP_BIRTH = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/HALeWHOregionLifeExpectancyAtBirth.csv')
HALE_EXP = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/ofHaleInLifeExpectancy.csv')
WHO_Life_Exp_Birth = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/lifeExpectancyAtBirth.csv')
MedicalDoctors = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/medicalDoctors.csv')
CancerEtc = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/30-70cancerChdEtc.csv')

UnsafeWash = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/mortalityRateUnsafeWash.csv')

MaternalMortality = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/maternalMortalityRatio.csv')

Sanitation = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/atLeastBasicSanitizationServices.csv') # Population using at least basic sanitation services (%)

UHC = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/uhcCoverage.csv')

Pharmacists = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/pharmacists.csv')

DrinkingWater = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/basicDrinkingWaterServices.csv') # Population using at least basic drinking-water services (%)

NeonatalMortality = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/neonatalMortalityRate.csv')

# 2. Data Cleansing

In [ ]:
def clean(df):
    df['Location'] = df['Location'].str.replace('United Kingdom of Great Britain and Northern Ireland', 'United Kingdom')
    df['Location'] = df['Location'].str.replace('Viet Nam', 'Vietnam')
    df['Location'] = df['Location'].str.replace('Democratic Republic of the Congo', 'Dem. Rep. Congo')

    df['Location'] = df['Location'].str.replace('Republic of Korea', 'South Korea')
    df['Location'] = df['Location'].str.replace("Democratic People's South Korea", 'North Korea')

    df['Location'] = df['Location'].str.replace("Lao People's Democratic Republic", 'Laos')
    df['Location'] = df['Location'].str.replace("Russian Federation", 'Russia')
    df['Location'] = df['Location'].str.replace("Equatorial Guinea", 'Eq. Guinea')
    df['Location'] = df['Location'].str.replace("Dominican Republic", 'Dominican Rep.')
    df['Location'] = df['Location'].str.replace("South Sudan", 'S. Sudan')
    df['Location'] = df['Location'].str.replace("Côte d’Ivoire", "Côte d'Ivoire")
    df['Location'] = df['Location'].str.replace("United Republic of Tanzania", 'Tanzania')
    df['Location'] = df['Location'].str.replace("Central African Republic", 'Central African Rep.')
    df['Location'] = df['Location'].str.replace("Syrian Arab Republic", 'Syria')
    df['Location'] = df['Location'].str.replace("The former Yugoslav Republic of Macedonia", 'Macedonia')
    df['Location'] = df['Location'].str.replace("Bosnia and Herzegovina", 'Bosnia and Herz.')
    df['Location'] = df['Location'].str.replace("Republic of Moldova", 'Moldova')
    df['Location'] = df['Location'].str.replace("Solomon Islands", 'Solomon Is.')
    df['Location'] = df['Location'].str.replace("Brunei Darussalam", 'Brunei')
    df['Location'] = df['Location'].str.replace("Eswatini", 'eSwatini')
    
    df.loc[df['Location'].str.startswith('Iran'), 'Location'] = 'Iran' 
    df.loc[df['Location'].str.startswith('Venezuela'), 'Location'] = 'Venezuela' 
    df.loc[df['Location'].str.startswith('Bolivia'), 'Location'] = 'Bolivia' 
    df.loc[df['Location'].str.startswith('Micronesia'), 'Location'] = 'Micronesia' 
    df.loc[df['Location'].str.startswith('Sudan'), 'Location'] = 'Sudan' 

NeonatalMortality2 = NeonatalMortality
WHO_Life_Exp_Birth2 = WHO_Life_Exp_Birth
MedicalDoctors2 = MedicalDoctors
CancerEtc2 = CancerEtc
MaternalMortality2 = MaternalMortality

clean(NeonatalMortality2)
clean(WHO_Life_Exp_Birth2)
clean(MedicalDoctors2)
clean(CancerEtc2)
clean(MaternalMortality2)
clean(NeonatalMortality2)

WHO_Life_Exp_Birth2.drop(WHO_Life_Exp_Birth2.loc[WHO_Life_Exp_Birth2['Period'] == 1920].index, inplace=True)



In [ ]:
MedicalDoctorsPivot = pd.pivot_table(MedicalDoctors2, values=['First Tooltip'], index=['Location'], columns = ['Period'])

MedicalDoctorsPivot2 = MedicalDoctorsPivot

for x in range(len(MedicalDoctorsPivot2.index)):
    for i in range(len(MedicalDoctorsPivot2.columns)):
        if np.isnan(MedicalDoctorsPivot2.iloc[x, i]):
            MedicalDoctorsPivot2.iloc[x, i] = MedicalDoctorsPivot2.iloc[x, (i-1)]        

#DoctorsFinal = []
#for x in range(29):
#    for y in range(194):
#        DoctorsFinal.append([MedicalDoctorsPivot2.index[y],
#                             MedicalDoctorsPivot2.columns[x],
#                             MedicalDoctorsPivot2.iloc[y,x]])
    
#DoctorsFinal = pd.DataFrame(DoctorsFinal,columns=["Location", "Period", 'Doctors'])     
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world = world.rename(columns={"name": "Location"})
#test4 = pd.merge(world,DoctorsFinal,on=['Location'],how='left')
#test4 = test4.dropna()
#test4 = test4.sort_values(by=['Period']).reset_index()

# Visualization

## Life Expectancy at Birth

### North and South America, Europe, Japan and Australia have high life expectancies, in contrast with Africa which has very low, some lower than 50 years.

In [ ]:
WHO_Life_Exp_Birth_Sex = WHO_Life_Exp_Birth2[WHO_Life_Exp_Birth2['Dim1'] == 'Both sexes'].sort_values(by=['Period']).reset_index()

world = world.rename(columns={"name": "Location"})

test = pd.merge(world,WHO_Life_Exp_Birth_Sex,on=['Location'],how='left')

test = test.dropna()

test = test.sort_values(by=['Period']).reset_index()

test.loc[test.Location == 'France', 'iso_a3'] = "FRA"
test.loc[test.Location == 'Norway', 'iso_a3'] = "NOR"

fig = px.choropleth(test, locations="iso_a3",
                    color="First Tooltip", # lifeExp is a column of gapminder
                    hover_name="Location",# column to add to hover information
                    animation_frame='Period',
                    color_continuous_midpoint = 60,
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1200, height=800,
                    title = 'Life Expectancy at Birth')

fig.update_traces(colorbar_nticks=10, selector=dict(type='choropleth'))
fig.show()

In [ ]:
WHO_Life_Exp_Birth2Pivot = pd.pivot_table(WHO_Life_Exp_Birth2, values=['First Tooltip'], index=['Dim1'], columns = ['Period'], aggfunc={'First Tooltip': np.mean}).T

WHO_Life_Exp_Birth2Pivot['Date'] = [2000, 2010, 2015, 2019]

fig = px.line(WHO_Life_Exp_Birth2Pivot, x="Date", y=["Both sexes", 'Male', 'Female'], title = ' Global Life Expectancy at Birth')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

## Doctors per 10,000 persons

### The highest numbers of doctors are seen in Europe, Russia and Kazakhstan, with Cuba having the highest number, however, African countries have the lowest numbers.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world.rename(columns={"name": "Location"})

test3 = pd.merge(world,MedicalDoctors2,on=['Location'],how='left')

test3 = test3.dropna()

test3 = test3.sort_values(by=['Period']).reset_index()


test3.loc[test3.Location == 'France', 'iso_a3'] = "FRA"
test3.loc[test3.Location == 'Norway', 'iso_a3'] = "NOR"

fig = px.choropleth(test3, locations="iso_a3",
                    color="First Tooltip", # lifeExp is a column of gapminder
                    color_discrete_map={
                        0-5: '#fffcfc',
                        5-10 : '#ffdbdb',
                        10-20 : '#ffbaba',
                        20-30 : '#ff9e9e',
                        30-40 : '#ff7373',
                        40-50 : '#ff4d4d',
                        50-90 : '#ff0d0d'},
                    hover_name="Location",# column to add to hover information
                    animation_frame='Period',
                    color_continuous_midpoint = 30,
                    
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1200, height=800,
                    title="Medical Doctors per 10,000 persons")

fig.update_traces(colorbar_nticks=10, selector=dict(type='choropleth'))
fig.show()


In [ ]:
MedicalDoctors2Pivot = pd.pivot_table(MedicalDoctors2, values=['First Tooltip'], index=['Period'], aggfunc={'First Tooltip': np.mean})
fig = px.line(MedicalDoctors2Pivot, x=MedicalDoctors2Pivot.index, y='First Tooltip', title = 'Doctors per 10,000 persons (global mean)')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

## Cardiovascular Disease, Cancer, Diabetes and Chronic Respiratory Disease

In [ ]:
# 'Probability (%) of dying between age 30 and exact age 70 from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={"name": "Location"})
test5 = pd.merge(world,CancerEtc2,on=['Location'],how='left')
test5 = test5.dropna()
test5 = test5.sort_values(by=['Period']).reset_index()
test5.loc[test5.Location == 'France', 'iso_a3'] = "FRA"
test5.loc[test5.Location == 'Norway', 'iso_a3'] = "NOR"
fig = px.choropleth(test5, locations="iso_a3",
                    color="First Tooltip", # lifeExp is a column of gapminder
                    hover_name="Location",# column to add to hover information
                    animation_frame='Period',
                    #color_continuous_midpoint = 60,
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1200, height=800,
                    title = "Probability (%) of dying between age 30 and exact age 70 from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease")
fig.update_traces(colorbar_nticks=10, selector=dict(type='choropleth'))
fig.show()

In [ ]:
CancerEtc2Pivot = pd.pivot_table(CancerEtc2, values=['First Tooltip'], index=['Dim1'], columns = ['Period'], aggfunc={'First Tooltip': np.mean}).T
CancerEtc2Pivot['Date'] = [2000, 2005, 2010, 2015, 2016]
fig = px.line(CancerEtc2Pivot, x="Date", y=["Both sexes", 'Male', 'Female'], title = 'Mean Global Probability (%) of dying between age 30 and exact age 70 from any of cardiovascular disease, cancer, diabetes, or chronic respiratory disease')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

# Maternal Mortality Ratio

### Maternal mortality refers to deaths due to complications from pregnancy or childbirth. Central African countries as well as Afghanistan have a steadily high maternal mortality ratio over the last two decades.

In [ ]:
MaternalMortality = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/maternalMortalityRatio.csv')
MaternalMortality2 = MaternalMortality
clean(MaternalMortality2)
for i in range(len(MaternalMortality2)):
    MaternalMortality2['First Tooltip'][i] = re.sub("[\(\[].*?[\)\]]", "", MaternalMortality2['First Tooltip'][i])
MaternalMortality2['First Tooltip'] = MaternalMortality2['First Tooltip'].astype('int')
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.rename(columns={"name": "Location"})
test6 = pd.merge(world,MaternalMortality2,on=['Location'],how='left')
test6 = test6.dropna()
test6 = test6.sort_values(by=['Period']).reset_index()
#test6['First Tooltip'] = test6['First Tooltip'].astype('int')
test6.loc[test6.Location == 'France', 'iso_a3'] = "FRA"
test6.loc[test6.Location == 'Norway', 'iso_a3'] = "NOR"
fig = px.choropleth(test6, locations="iso_a3",
                    color="First Tooltip", # lifeExp is a column of gapminder
                    hover_name="Location",# column to add to hover information
                    animation_frame='Period',
                    color_continuous_midpoint = 800,
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1200, height=800,
                    title = 'Maternal mortality ratio (per 100 000 live births)')
fig.update_coloraxes(cmin=0, cmax=2000)                    
fig.update_traces(colorbar_nticks=10, selector=dict(type='choropleth'))
fig.show()

### Still, the global mean maternal mortality ratio is steadily decreasing.

In [ ]:
MaternalMortality2pivot = pd.pivot_table(MaternalMortality2, values=['First Tooltip'], columns = ['Period'], aggfunc={'First Tooltip': np.mean}).T
MaternalMortality2pivot
fig = px.line(MaternalMortality2pivot, x=MaternalMortality2pivot.index, y=['First Tooltip'], title = 'Maternal mortality ratio (per 100 000 live births)')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

# Neonatal Mortality 

### Neonatal mortality refers to death of a live-born baby within the first 28 days of life. In this case it is measured per 1,000 live bearths

In [ ]:
NeonatallMortality = NeonatalMortality = pd.read_csv('/kaggle/input/who-worldhealth-statistics-2020-complete/neonatalMortalityRate.csv')
NeonatalMortality2 = NeonatalMortality
clean(NeonatalMortality2)

for i in range(len(NeonatalMortality2)):
    NeonatalMortality2['First Tooltip'][i] = re.sub("[\(\[].*?[\)\]]", "", NeonatalMortality2['First Tooltip'][i])

NeonatalMortality2['First Tooltip'] = NeonatalMortality2['First Tooltip'].astype('float64')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

world = world.rename(columns={"name": "Location"})

test7 = pd.merge(world, NeonatalMortality2,on=['Location'],how='left')

test7 = test7.dropna()

test7 = test7.sort_values(by=['Period']).reset_index()

#test6['First Tooltip'] = test6['First Tooltip'].astype('int')

test7.loc[test7.Location == 'France', 'iso_a3'] = "FRA"
test7.loc[test7.Location == 'Norway', 'iso_a3'] = "NOR"

fig = px.choropleth(test7, locations="iso_a3",
                    color="First Tooltip", # lifeExp is a column of gapminder
                    hover_name="Location",# column to add to hover information
                    animation_frame='Period',
                    #color_continuous_midpoint = 800,
                    color_continuous_scale=px.colors.sequential.Plasma,
                    width=1200, height=800,
                    title = 'Neonatal mortality rate (per 1000 live births)')

fig.update_coloraxes(cmin=0, cmax=80)                    
fig.update_traces(colorbar_nticks=10, selector=dict(type='choropleth'))
fig.show()

In [ ]:
NeonatalMorPivot = pd.pivot_table(NeonatalMortality2, values=['First Tooltip'], columns = ['Period'], aggfunc={'First Tooltip': np.mean}).T

fig = px.line(NeonatalMorPivot, x=NeonatalMorPivot.index, y=['First Tooltip'], title = 'Global Mean Neonatal mortality rate (per 1000 live births)')
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.update_layout(hovermode="x")
fig.show()

In [ ]:
test7
test7pivot = pd.pivot_table(test7, values=['First Tooltip'], index=['Period','continent'], aggfunc={'First Tooltip': np.sum})
test3pivot = pd.pivot_table(test3, values=['First Tooltip'], index=['Period','continent'], aggfunc={'First Tooltip': np.sum})
test7pivot = test7pivot.rename(columns={"First Tooltip": "Neonatal Mortality"})
test3pivot = test3pivot.rename(columns={"First Tooltip": "Doctors per 10,000"})
testmerge = pd.merge(test3pivot, test7pivot, on=['Period', 'continent'], how='left')

testmerge['indx'] = testmerge.index

testmerge.indx = testmerge.indx.str.replace('[^a-zA-Z0-9]', '')
testmerge

#testmerge.unstack(level='Period')
#df.loc[df['Location'].str.startswith('Iran'), 'Location'] = 'Iran' 

#px.scatter(testmerge, x="Doctors per 10,000", y="Neonatal Mortality", animation_frame="Period", color="continent")

In [ ]:
#test7 Neonatal Mortality rate
#test3 Doctors per 10 000

#test7pivot = pd.pivot_table(test7, values=['First Tooltip', 'Period'], columns = ['continent'], aggfunc={'First Tooltip': np.sum}).T
test7pivot = test7pivot.rename(columns={"First Tooltip": "Neonatal Mortality"})

test3pivot = pd.pivot_table(test3, values=['First Tooltip'], columns = ['continent', 'Period'], aggfunc={'First Tooltip': np.sum}).T
test3pivot = test3pivot.rename(columns={"First Tooltip": "Doctors per 10,000"})

testmerge = pd.merge(test3pivot,test7pivot,on=['continent', 'Period'],how='left')
testmerge
test7
#fig = px.scatter(testmerge, x="Doctors per 10,000", y="Neonatal Mortality", animation_frame="Period", animation_group="continent", color="continent", hover_name="location")

In [ ]:
fig = px.scatter(df, x="total_cases", y="total_deaths", animation_frame="date", animation_group="location",
            size="population", color="continent", hover_name="location",
            log_x=True, log_y=True, size_max=45, range_x=[100,10000000], range_y=[25,140000])

# ************ WORK IN PROGRESS ************